In [64]:
# import pandas lib as pd
import pandas as pd
import numpy as np
# read by default 1st sheet of an excel file
df= pd.read_excel('datatest.xlsx',header=0, sheet_name="Style1")
row_name=df.columns.values
df.insert(2,'sewing',0,True)
df.insert(3,'wating',df['headcount'],True)

In [65]:
# import pandas lib as pd
import pandas as pd
# read by default 1st sheet of an excel file
df2= pd.read_excel('datatest.xlsx',header=0, sheet_name="Style2")
row_name=df2.columns.values
df2.insert(2,'sewing',0,True)
df2.insert(3,'wating',df2['headcount'],True)
df2

,sequence,operation,sewing,wating,SAM,headcount,WIP
0,1,Bind Panel,0,4,3.3,4,7
1,2,Over Edge,0,4,3.6,4,25
2,3,Attach Sling Rubber,0,7,6.0,7,10
3,4,Sew Center,0,8,8.0,8,14
4,5,Topstitch,0,6,4.0,6,8
5,6,Seam Panel,0,20,15.0,20,18
6,7,Seam Crotch,0,7,7.4,7,10
7,8,Hem Legs,0,8,8.6,8,6
8,9,Sew Band,0,9,9.2,9,0


In [66]:
# import pandas lib as pd
import pandas as pd
# read by default 1st sheet of an excel file
df3= pd.read_excel('datatest.xlsx',header=0, sheet_name="OPERATION_CONFIG")
row_name=df3.columns.values
df3

,style,operation,next operation
0,Style 1,Bind Panel,Seam Panel
1,Style 2,Bind Panel,Sew Center
2,Style 2,Over Edge,Sew Center
3,Style 2,Sew Center,Topstitch
4,Style 2,Attach Sling Rubber,Topstitch


In [67]:
# import pandas lib as pd
import pandas as pd
# read by default 1st sheet of an excel file
df4= pd.read_excel('datatest.xlsx',header=0, sheet_name="NO_WIP_REQ")
row_name=df4.columns.values
df4

,style,operation
0,Style 1,Bind Panel
1,Style 1,Over Edge
2,Style 2,Bind Panel
3,Style 2,Over Edge
4,Style 2,Attach Sling Rubber


In [71]:

def update_sequence(df_style,point_check):
    sequence=point_check[1]
    sew_collect=point_check[2]
    df_style.loc[sequence-1,'sewing']-=sew_collect
    df_style.loc[sequence-1,'wating']+=sew_collect
    df_style.loc[sequence-1,'WIP']+=sew_collect
    
def cal_time_check(df_style,time,num_seq,sew,list_point_check):
    output=[]
    point_time=time + df_style.loc[num_seq-1,'SAM']
    output.append(round(point_time,1))
    output.append(num_seq)
    output.append(sew)
    list_point_check.append(output)
    list_point_check.sort()
    return list_point_check
    
def sequence_first(df_style,ti_step,ti_line,num_seq,list_point_check):
    df_style.loc[num_seq-1,'wating']=0
    df_style.loc[num_seq-1,'sewing']=df_style.loc[num_seq-1,'headcount']
    cycle=ti_line//df_style.loc[num_seq-1,'SAM']
    if ti_step==0:
        for cyc in range(0,int(cycle)):
            cal_time_check(df_style,cyc*df_style.loc[num_seq-1,'SAM'],num_seq,df_style.loc[num_seq-1,'headcount'],list_point_check)
            # print(time)
            # print(".") 
    # print(time)
    # print(df['WIP'][0])
    
def cal_wip(df_style,num_seq,time,list_point_check,list_seq_first,list_num_seq_dep,list_seq_depend):
    name_seq=df_style.loc[num_seq-1,"operation"]
    if name_seq in list_seq_first:
        sequence_first(df_style,0.1,0.1,num_seq,list_point_check)
    elif name_seq in list_seq_depend:
        for item in list_num_seq_dep:
            if num_seq == item.pop(0):
                min_wip=df_style.loc[item-1]["WIP"].min()
                if df_style.loc[num_seq-1,'wating']<=min_wip:
                    cal_time_check(df_style,time,num_seq,df_style.loc[num_seq-1,'wating'],list_point_check)
                    df_style.loc[num_seq-1,'sewing']+=df_style.loc[num_seq-1,'headcount']
                    df_style.loc[num_seq-1,'wating']=0
                    for i in item:
                        df_style.loc[i-1,"WIP"]-=df_style.loc[num_seq-1,'wating']
                if df_style.loc[num_seq-1,'wating']>=min_wip and min_wip >0:
                    cal_time_check(df_style,time,num_seq,min_wip,list_point_check)
                    df_style.loc[num_seq-1,'sewing']+=min_wip
                    df_style.loc[num_seq-1,'wating']-=min_wip
                    for i in item:
                        df_style.loc[i-1,"WIP"]-=min_wip
    else:   
        if df_style.loc[num_seq-1,'wating'] > 0 and df_style.loc[num_seq-2,'WIP'] > 0 and df_style.loc[num_seq-2,'WIP'] <= df_style.loc[num_seq-1,'wating']:
            cal_time_check(df_style,time,num_seq,df_style.loc[num_seq-2,'WIP'],list_point_check)
            df_style.loc[num_seq-1,'wating']-=df_style.loc[num_seq-2,'WIP']
            df_style.loc[num_seq-1,'sewing']+=df_style.loc[num_seq-2,'WIP']
            df_style.loc[num_seq-2,'WIP']=0

        if df_style.loc[num_seq-1,'wating']> 0 and df_style.loc[num_seq-2,'WIP']> df_style.loc[num_seq-1,'wating']:
            cal_time_check(df_style,time,num_seq,df_style.loc[num_seq-1,'wating'],list_point_check)
            df_style.loc[num_seq-2,'WIP']-=df_style.loc[num_seq-1,'wating']
            df_style.loc[num_seq-1,'wating']=0
            df_style.loc[num_seq-1,'sewing']=df_style.loc[num_seq-1,'headcount']
        


def get_sequence_first(df_no_wip_req, name_style):
    list_seq_first=df_no_wip_req[df_no_wip_req["style"]==name_style]["operation"].values
    return list_seq_first

def get_sequence_depend(df_config, name_style):
    list_seq_previous=[]
    df_=df_config[df_config["style"]==name_style]
    list_seq_depend=df_["next operation"].unique()    
    for i in list_seq_depend:
        a=df_config[df_config["next operation"]==i]["operation"].values
        list_seq_previous.append(a)
    return list_seq_depend, list_seq_previous

def check_sequence(df_style,list_seq_first,list_seq_depend,list_seq_previous,ti_line,num_seq,list_point_check,list_num_seq_dep):
    name_seq=df_style.loc[num_seq-1,"operation"]
    item_dep=[]
    if name_seq in list_seq_first:
        sequence_first(df_style,0,ti_line,num_seq,list_point_check)
    if name_seq in list_seq_depend:
        item_dep.append(num_seq)
        index=np.where(list_seq_depend==name_seq)
        for i in list_seq_previous[index]:
            item_dep.append(df_style[df_style["operation"==i]]["sequence"])
        list_num_seq_dep.append(item_dep)
    

if __name__=="__main__":
    list_check=[]
    time_step=0
    time_line=100
    list_first= get_sequence_first(df4,"Style 1")
    list_depend,list_previous=get_sequence_depend(df3,"Style 1")
    lis_num_se=[]
    
    for i in range(1,df.shape[0]+1):
        check_sequence(df,list_first,list_depend,list_previous,time_line,i,list_check,lis_num_se)
        cal_wip(df,i,time_step,list_check,list_first,lis_num_se,list_depend)
    # sequence_first(df,time_step,time_line,1,list_check)
    # print(time_step)
    # print(df)
    # for _ in list_check:
    #     print(_)
    pcheck=list_check.pop(0)
    time_step=pcheck[0]
    # print(pcheck)
    # print(time_step)
    while time_step<= time_line:
        update_sequence(df,pcheck)
        # sequence_first(df,time_step,time_line,1,list_check)
        
        for i in range(1,df.shape[0]+1):
            cal_wip(df,i,time_step,list_check,list_first,lis_num_se,list_depend)
        # print(time_step)
        # print(df)
        # for _ in list_check:
        #     print(_)
        try:
            pcheck=list_check.pop(0)
            time_step=pcheck[0]
        except:
            break
        # print(pcheck)
        # print(time_step)
print(time_line)
print(df)

TypeError: list indices must be integers or slices, not tuple